# Training evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import random
import math
import re
import time
import tensorflow as tf
import matplotlib
import matplotlib.patches as patches
import json
import skimage.io
from IPython.display import clear_output
import numpy as np

import utils
import visualize
from visualize import display_images
import model as modellib
from model import log
import xavi
import coco

ROOT_DIR = os.getcwd() # Root directory of the project
XAVI_DIR = os.path.join(ROOT_DIR, "XAVI_Dataset") # Xavi Dataset directory
MODEL_DIR = os.path.join(XAVI_DIR, "model") # Directory to save trained model
DEFAULT_LOGS_DIR = os.path.join(MODEL_DIR, "logs") # Directory to save logs
XAVI_MODELS = os.path.join(DEFAULT_LOGS_DIR, "xavi51220210430T1046") # Local path to trained weights file
TRAINING_SUBSET = "train512"
VALIDATION_SUBSET = "val512"
COCO_DIR = os.path.join(os.getcwd(), "COCO_Dataset") # Xavi Dataset directory
COCO_VALIDATION_SUBSET = "valminusminival"

matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline 

Using TensorFlow backend.


## Load datasets

### XAVI dataset

In [3]:
# Build validation dataset
json_categories = json.load(open(os.path.join(XAVI_DIR, VALIDATION_SUBSET, "categories.json")))["Categories"]
xavi_dataset = xavi.XaviDataset()
xavi_dataset.load_xavi(XAVI_DIR, VALIDATION_SUBSET, xavi.MasksType["PNG"], json_categories, (512, 512))
xavi_dataset.prepare()

print("Images: {}\nClasses: {}".format(len(xavi_dataset.image_ids), xavi_dataset.class_names))

Images: 9870
Classes: ['BG', 'Human', 'Vehicle_Car']


### COCO dataset

In [4]:
# Build validation dataset
coco_dataset = coco.CocoDataset()
coco_dataset.load_coco(COCO_DIR, COCO_VALIDATION_SUBSET, class_ids=[1,3])
coco_dataset.prepare()

clear_output(wait=True)
print("Images: {}\nClasses: {}".format(len(coco_dataset.image_ids), coco_dataset.class_names))

Images: 20129
Classes: ['BG', 'person', 'car']


## Evaluation of the training

For each class, for each model (output of each epoch) and each dataset, compute the mAP:

In [8]:
N_IMAGES = 100

# Load config and model in inference mode
config = xavi.XaviConfig(json_categories)
xavi_model = modellib.MaskRCNN(mode="inference", model_dir=DEFAULT_LOGS_DIR, config=config)

APS_XaviDataset = []
APS_XaviDataset_humans = []
APS_XaviDataset_cars = []

APS_CocoDataset = []
APS_CocoDataset_humans = []
APS_CocoDataset_cars = []

weights_files = [f for f in os.listdir(XAVI_MODELS) if f.endswith(".h5")]

for i, weights_file in enumerate(weights_files):
    
    # Load model
    xavi_model.load_weights(os.path.join(XAVI_MODELS, weights_file), by_name=True)
    
    # Pick random images from validation test
    xavi_ids = np.random.choice(xavi_dataset.image_ids, N_IMAGES, replace=False)
    coco_ids = np.random.choice(coco_dataset.image_ids, N_IMAGES, replace=False)
    
    # Compute AP for XAVI Dataset
    (APs, class_APs) = utils.compute_batch_ap(xavi_dataset, config, xavi_model, xavi_ids, show_images = False, target_classes=(1,2))
    APS_XaviDataset.append(np.mean(APs))
    APS_XaviDataset_humans.append(np.mean(class_APs[1]))
    APS_XaviDataset_cars.append(np.mean(class_APs[2]))
    
    # Compute AP for COCO Dataset
    (APs, class_APs) = utils.compute_batch_ap(coco_dataset, config, xavi_model, coco_ids, show_images = False, target_classes=(1,2))
    APS_CocoDataset.append(np.mean(APs))
    APS_CocoDataset_humans.append(np.mean(class_APs[1]))
    APS_CocoDataset_cars.append(np.mean(class_APs[2]))
    
    clear_output(wait=True)
    print(i, " / ", len(weights_files))
    print("APS Xavi Dataset: ", APS_XaviDataset)
    print("APS COCO Dataset: ", APS_CocoDataset)

clear_output(wait=True)
print("APS Xavi Dataset: ", APS_XaviDataset)
print("APS Xavi Dataset (humans): ", APS_XaviDataset_humans)
print("APS Xavi Dataset (cars): ", APS_XaviDataset_cars)
print("APS COCO Dataset: ", APS_CocoDataset)
print("APS COCO Dataset (humans): ", APS_CocoDataset_humans)
print("APS COCO Dataset (cars): ", APS_CocoDataset_cars)

39  /  40
APS Xavi Dataset:  [0.0, 0.016297619181374706, 0.16406865214763414, 0.23908207234645645, 0.26184648754483175, 0.34638412870218355, 0.5171683294397025, 0.44260648389349855, 0.5002865115238561, 0.5233369076482597, 0.5251807392608552, 0.5735572128693263, 0.5446538910681293, 0.6468238132631495, 0.6684801622423387, 0.6271978441236371, 0.6418244731533858, 0.6142526106032588, 0.6292948264620747, 0.6682607743484632, 0.6942916684498391, 0.6606174643722673, 0.706030680836505, 0.7098508627040994, 0.6795901911613487, 0.7321865118674579, 0.7191466295087621, 0.7069950824633596, 0.682114774895211, 0.6901150817631019, 0.7412405396650236, 0.7208477670085571, 0.7515952404872291, 0.7734602120144973, 0.7768703740860025, 0.8002589310707435, 0.7702603210934571, 0.7824582477178837, 0.796106641292572, 0.8157924622711208]
APS COCO Dataset:  [0.0, 0.0019117965859671432, 0.0606681933812797, 0.04823266775254563, 0.035912271407831986, 0.09487523627600498, 0.1225077024286594, 0.09069658160363397, 0.162447